### 0. `torch.nn.DataParallel`

Pytorch provides multi-gpu settings by `torch.nn.DataParallel` module and this works like the below figures at forward and backward:

<img src='./imgs/forward1.png' width='500'>
<img src='./imgs/forward2.png' width='500'>
<center><span style="color:gray"><em>from https://pytorch.org/tutorials/beginner/former_torchies/parallelism_tutorial.html/</em></span></center>

Within the forward process, each of multiple gpus should have model and data. This process is taken by the following four steps:

* `scatter`: batch is splited into multiple mini-batches
    * the size of mini-batch is calcuated by dividing batch size with the number of gpus
    
* `replicate`: the model is replicated in each of multiple gpus

* `parallel forward`: each gpus(model) go through forward

* `gather`: each output tensor of multiple gpus is collected to a speicified gpu

<img src='./imgs/backward1.png' width='500'>
<img src='./imgs/backward2.png' width='500'>
<center><span style="color:gray"><em>from https://pytorch.org/tutorials/beginner/former_torchies/parallelism_tutorial.html/</em></span></center>

Normally, loss function is assigned to calculate loss and it is used to do back-propagation/backward. 
 Back-propagation은 각 GPU에서 수행하며 그 결과로 각 GPU에 있던 모델의 gradient를 구할 수 있습니다. 만약 4개의 GPU를 사용한다면 4개의 GPU에 각각 모델이 있고 각 모델은 계산된 gradient를 가지고 있습니다. 이제 모델을 업데이트하기 위해 각 GPU에 있는 gradient를 또 하나의 GPU로 모아서 업데이트를 합니다. 만약 Adam과 같은 optimizer를 사용하고 있다면 gradient로 바로 모델을 업데이트하지 않고 추가 연산을 합니다. 이러한 Data Parallel 기능은 다음과 같이 코드 한 줄로 간단히 사용 가능합니다.

```python
import torch
import torch.nn as nn

model = MODEL(args)
model = torch.nn.DataParallel(model)
model.cuda()

...

for inputs, labels in train_loader:
    outputs = model(inputs)          
    loss = criterion(outputs, labels)     
    
    optimizer.zero_grad()
    loss.backward()                        
    optimizer.step()
```

`nn.DataParallel`로 model을 감싸면 학습을 할 때 다음과 같은 작업을 하는 것입니다. 위에서 언급한 대로 `replicate` → `scatter` → `parallel_apply` → `gather` 순서대로 진행합니다. 

```python
def data_parallel(module, input, device_ids, output_device):
    replicas = nn.parallel.replicate(module, device_ids)
    inputs = nn.parallel.scatter(input, device_ids)
    replicas = replicas[:len(inputs)]
    outputs = nn.parallel.parallel_apply(replicas, inputs)
    return nn.parallel.gather(outputs, output_device)
```

### 1. Imablanced memory in use 

<img src='./imgs/memory_imbalance.png'>

multi-gpus를 활용한다고 하더라도, loss gradient의 계산은 하나의 gpu에 결과값들을 모은 후 이루어지기 때문에 해당하는 gpu의 메모리의 사용량은 클 수밖에 없다.

#### Python-Encoding

[PyTorch-Encoding](https://github.com/zhanghang1989/PyTorch-Encoding)은 `DataParallel`을 그대로 사용하면서 메모리 불균형의 문제를 해결할 수 있는 패키지입니다. 모델은 `DataParallel`을 통해 병렬로 연산할 수 있게 만들었지만 loss function이 그대로이기 때문에 하나의 GPU에서 loss를 계산합니다. 따라서 loss function 또한 병렬로 연산하도록 만든다면 메모리 불균형 문제를 어느정도 해결할 수 있습니다.

PyTorch-Encoding 중에서도 다음 파이썬 코드에 loss function을 parallel하게 만드는 코드가 들어있습니다.

https://github.com/zhanghang1989/PyTorch-Encoding/blob/master/encoding/parallel.py

PyTorch에서는 loss function 또한 하나의 모듈이므로 각 GPU에 `replicate`할 수 있습니다. 그리고 데이터의 정답(target)에 해당하는 tensor를 각 GPU로 `scatter` 하여 각 GPU에서 loss 값을 계산할 수 있도록 합니다. 따라서, 각 GPU에서는 계산한 loss로 바로 backward 연산을 할 수 있습니다.

<img src='./imgs/loss.png' width='1200'>

Loss function을 parallel 하게 만들어서 연산하는 과정을 코드로 보자면 다음과 같습니다. 데이터의 정답에 해당하는 target을 scatter 한 다음에 replicate한 module에서 각각 계산을 합니다. 계산한 output와 Reduce.apply를 통해 각 GPU에서 backward 연산을 하도록 만듭니다.

```python
from torch.nn.parallel.data_parallel import DataParallel

class DataParallelCriterion(DataParallel):
    def forward(self, inputs, *targets, **kwargs):
        targets, kwargs = self.scatter(targets, kwargs, self.device_ids)
        replicas = self.replicate(self.module, self.device_ids[:len(inputs)])
        targets = tuple(targets_per_gpu[0] for targets_per_gpu in targets)
        outputs = _criterion_parallel_apply(replicas, inputs, targets, kwargs)
        return Reduce.apply(*outputs) / len(outputs), targets
```

다음과 가티 Pytorch-Encoding 패키지에서 parallel.py 파일만 가져와서 학습 코드에서 import 하고, `DataParallelModel`과 `DataParallelCriterion`을 사용여 학습을 하면 됩니다.

```python
import torch
import torch.nn as nn
from parallel import DataParallelModel, DataParallelCriterion

model = MODEL(args)
model = DataParallelModel(model)
model.cuda()

criterion = nn.NLLLoss()
criterion = DataParallelCriterion(criterion) 

...

for i, (inputs, labels) in enumerate(trainloader):
    outputs = model(inputs)          
    loss = criterion(outputs, labels)     
    
    optimizer.zero_grad()
    loss.backward()                        
    optimizer.step()
```

이렇게 학습을 할 경우에 Nvidia-smi 출력 결과는 다음과 같습니다. batch size 는 200으로 동일합니다. DataParallel 만 사용할 때에 비해 1번 GPU와 2번 GPU의 메모리 사용량의 차이가 상당히 줄었습니다. batch size를 기존에 비해 늘릴 수 있기 때문에 학습 시간도 전체적으로 1/3 정도가 줄었습니다. 하지만 GPU-Util의 수치로 확인할 수 있듯이 GPU 성능을 여전히 제대로 활용 못하고 있습니다. GPU 성능을 100 %로 끌어 올리려면 어떻게 해야할까요?

<img src='./imgs/memory_balance.png'>

### 2. Use GPU-Util 100%

#### 2-1.
분산 학습 자체는 하나의 컴퓨터로 학습하는게 아니라 여러 컴퓨터를 사용해서 학습하는 경우를 위해 개발된 것이지만, multi-GPU 학습을 할 때도 분산 학습을 사용할 수 있습니다. 그리고 [PyTorch에서는 `DataParallel`과 함께 분산 학습과 관련된 기능을 제공합니다.](https://pytorch.org/tutorials/intermediate/dist_tuto.html)

다른 예로는 ImageNet에 딥러닝 모델을 학습시키는 코드 예제입니다. 이 예제에서 여러 머신에서 분산 학습을 하는 방법을 소개하는데 하나의 머신에서 여러 GPU 학습하는 방법도 소개합니다.

https://github.com/pytorch/examples/blob/master/imagenet/main.py

ImageNet 예제의 main.py 에서 multi-GPU와 관련된 주요 부분을 다음과 같이 정리해 봤습니다. main.py를 실행하면 main이 실행되는데 main은 다시 main_worker 들을 multi-processing으로 실행합니다. GPU 4개를 하나의 노드로 보고 world_size를 설정합니다. 그러면 mp.spawn 함수가 4개의 GPU에서 따로 따로 main_worker를 실행합니다.

```python
import torch.distributed as dist
from torch.nn.parallel import DistributedDataParallel


def main():
    args = parser.parse_args()

    ngpus_per_node = torch.cuda.device_count()
    args.world_size = ngpus_per_node * args.world_size
    mp.spawn(main_worker, nprocs=ngpus_per_node, 
             args=(ngpus_per_node, args))
    
    
def main_worker(gpu, ngpus_per_node, args):
    global best_acc1
    args.gpu = gpu
    torch.cuda.set_device(args.gpu)
    
    print("Use GPU: {} for training".format(args.gpu))
    args.rank = args.rank * ngpus_per_node + gpu
    dist.init_process_group(backend='nccl', 
                            init_method='tcp://127.0.0.1:FREEPORT',
                            world_size=args.world_size, 
                            rank=args.rank)
    
    model = Bert()
    model.cuda(args.gpu)
    model = DistributedDataParallel(model, device_ids=[args.gpu])

    acc = 0
    for i in range(args.num_epochs):
        model = train(model)
        acc = test(model, acc)
```

main_worker에서 dist.init_process_group을 통해 각 GPU 마다 분산 학습을 위한 초기화를 실행합니다. PyTorch의 docs를 보면 multi-GPU 학습을 할 경우 backend로 nccl을 사용하라고 나와있습니다. init_method에서 FREEPORT에 사용 가능한 port를 적으면 됩니다. 이렇게 분산 학습을 위한 초기화를 하고 나면 분산 학습이 가능합니다. 28번째 줄을 보면 model에는 DataParallel 대신에 DistributedDataParallel을 사용하는 것을 볼 수 있습니다. DataParallel에서 언급한 입력을 분산하고 forward 연산을 수행하고 다시 backward 연산을 수행하는 역할을 합니다.

https://pytorch.org/docs/stable/distributed.html

DataLoader가 입력을 각 프로세스에 전달하기 위해서 다음처럼 DistributedSampler를 사용합니다. DistributedSampler는 DistributedDataParallel과 함께 사용해야 합니다. 사용 방법은 간단하게 정의해놓은 dataset를 DistributedSampler로 감싸주고 DataLoader에서 sampler에 인자로 넣어줍니다. 그 다음엔 평소에 DataLoader를 사용하듯이 똑같이 사용하면 됩니다.

```python
from torch.utils.data.distributed import DistributedSampler

train_dataset = datasets.ImageFolder(traindir, ...)
train_sampler = DistributedSampler(train_dataset)

train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=args.batch_size, shuffle=False,
    num_workers=args.workers, pin_memory=True, sampler=train_sampler)
```


DistributedSampler의 내부를 살짝 보자면 다음 코드와 같습니다(많은 부분을 생략했습니다). 각 Sampler는 전체 데이터를 GPU의 개수로 나눈 부분 데이터에서만 데이터를 샘플링합니다. 부분 데이터를 만들기 위해 전체 데이터셋 인덱스 리스트를 무작위로 섞은 다음에 그 인덱스 리스트를 쪼개서 각 GPU Sampler에 할당합니다. epoch 마다 각 GPU sampler에 할당되는 인덱스 리스트는 다시 무작위로 달라집니다. 그러기 위해서는 train_sampler.set_epoch(epoch) 명령어를 매 epoch 마다 학습 전에 실행해야 합니다.

```python
class DistributedSampler(Sampler):
    def __init__(self, dataset, num_replicas=None, rank=None):
        num_replicas = dist.get_world_size()
        rank = dist.get_rank()
        self.dataset = dataset
        self.num_replicas = num_replicas
        self.rank = rank
        self.num_samples = int(math.ceil(len(self.dataset) * 1.0 / self.num_replicas))
        self.total_size = self.num_samples * self.num_replicas
        
    def __iter__(self):
        g = torch.Generator()
        g.manual_seed(self.epoch)
        indices = torch.randperm(len(self.dataset), generator=g).tolist()
        indices = indices[self.rank:self.total_size:self.num_replicas]
        return iter(indices)
```

PyTorch Distributed 패키지를 사용해서 BERT 작은 모델을 학습해봤습니다. Nvidia-smi 를 통해 확인한 GPU 메모리 사용 현황은 다음과 같습니다. GPU 메모리 사용량이 완전 동일한 것을 볼 수 있습니다. 또한 GPU-Util의 수치도 99%로 상당히 높은 것을 볼 수 있습니다. 여기까지 왔다면 multi-GPU 학습을 제대로 할 준비가 됐습니다.

하지만 Distibuted DataParallel의 경우 학습을 시작하려 할 때 간간히 문제가 발생할 수 있습니다. 다음 github issue 글이 여러 문제 중에 하나를 보여줍니다. BERT 코드를 돌릴 때도 에러가 발생했는데 모델에서 학습에 사용하지 않는 parameter가 있을 경우에 Distributed DataParallel이 문제를 일으킬 수 있다는 의견이 있습니다. 이러한 문제를 신경쓰지 않고 학습을 하기 위해서 찾아보다가 Nvidia에서 만든 Apex라는 패키지를 발견했습니다.

https://github.com/facebookresearch/maskrcnn-benchmark/issues/318

#### 2-2. Nvidia Apex

Nvidia에서 Apex라는 Mixed Precision 연산을 위한 패키지를 만들었습니다. 보통 딥러닝은 32 비트 연산을 하는데 16 비트 연산을 사용해서 메모리를 절약하고 학습 속도를 높이겠다는 의도로 만든 것입니다. Apex에는 Mixed Precision 연산 기능 말고도 Distributed 관련 기능이 포함합니다. 이 포스트에서는 Mixed Precision에 대한 내용은 다루지 않습니다.
Apex의 Distributed DataParallel 기능을 하는 것이 DDP 입니다. Apex에서 ImageNet 학습을 위해 만든 예제에 관련 내용이 있습니다. Apex 사용법은 Docs에 잘 나와있으니 살펴보시면 됩니다.

https://github.com/NVIDIA/apex/blob/master/examples/imagenet/main_amp.py

다음 코드 2번 줄에서 보듯이 apex에서 DistributedDataParallel을 import 해서 사용합니다. 위 PyTorch 공식 예제에서와는 달리 코드 내에서 멀티 프로세싱을 실행하지 않습니다. 19 줄에서 보듯이 DDP로 model을 감싸줍니다. 그 이외에는 PyTorch DistributedDataParallel과 동일합니다.

```python
import torch.distributed as dist
from apex.parallel import DistributedDataParallel as DDP


def main():
    global args
    
    args.gpu = 0
    args.world_size = 1
    
    args.gpu = args.local_rank
    torch.cuda.set_device(args.gpu)
    torch.distributed.init_process_group(backend='nccl',
                                         init_method='env://')
    args.world_size = torch.distributed.get_world_size()
    
    model = Bert()
    model.cuda(args.gpu)
    model = DDP(model, delay_allreduce=True)

    acc = 0
    for i in range(args.num_epochs):
        model = train(model)
        acc = test(model, acc)
```

이 코드를 실행할 때는 다음 명령어를 사용해서 실행합니다. Torch.distributed.launch를 통해 main.py를 실행하는데 노드에서 4개의 프로세스가 돌아가도록 설정합니다. 각 프로세스는 GPU 하나에서 학습을 진행합니다. 만약 GPU가 2개라면 nproc_per_node를 2로 수정하면 됩니다. main.py에 batch_size와 num_worker를 설정하는데 각 GPU 마다의 batch_size와 worker 수를 의미합니다. batch size가 60이고 worker의 수가 2라면 전체적으로는 batch size가 240이며 worker의 수는 8입니다.

```
python -m torch.distributed.launch --nproc_per_node=4 main.py \
    --batch_size 60 \
    --num_workers 2 \
    --gpu_devices 0 1 2 3\
    --distributed \
    --log_freq 100 
```

Nvidia Apex를 사용해서 multi-GPU 학습을 했습니다. GPU 사용 현황은 다음과 같습니다. GPU 메모리 사용량이 모든 GPU에서 일정합니다.(3번 GPU는 다른 작업이 할당받고 있기 때문에 잡혀있습니다). GPU-Util을 보면 99% 아니면 100 %인 것을 알 수 있습니다.

DataParallel은 PyTorch에서 제공하는 가장 기본적인 방법이지만 GPU 메모리 불균형 문제가 생겼습니다. Custom DataParallel의 경우 GPU 메모리 문제를 어느정도 해결해주지만 GPU를 제대로 활용하지 못한다는 문제가 있었습니다. Distributed DataParallel은 원래 분산학습을 위해 만들어진 PyTorch의 기능이지만 multi-GPU 학습에도 사용할 수 있고 메모리 불균형 문제와 GPU를 활용하지 못하는 문제가 없었습니다. 하지만 간간히 문제가 발생하기 때문에 Nvidia에서 만든 Apex를 이용해서 multi-GPU 학습하는 것을 살펴봤습니다.
그렇다면 Apex를 사용하는 것이 항상 좋을까요? 제가 살펴본 이런 문제들이 딥러닝 학습을 할 때 항상 발생하지 않습니다. 만약 이미지 분류를 학습한다면 DataParallel 만으로 충분할 수 있습니다. BERT에서 GPU 메모리 불균형 문제가 생기는 이유는 모델 출력이 상당히 크기 때문입니다. 각 step마다 word의 개수만큼이 출력으로 나오기 때문에 이런 문제가 생깁니다. 하지만 이미지 분류의 경우 모델 자체가 클 수는 있어도 모델 출력은 그렇게 크지 않습니다. 따라서 GPU 메모리 불균형은 거의 없습니다.

https://discuss.pytorch.org/t/dataparallel-imbalanced-memory-usage/22551

https://discuss.pytorch.org/t/multi-gpu-training-memory-usage-in-balance/4163/4

https://medium.com/huggingface/training-larger-batches-practical-tips-on-1-gpu-multi-gpu-distributed-setups-ec88c3e51255

https://github.com/dnddnjs/pytorch-multigpu